# Interactive RCL temperture data

Experimental

In [ ]:
import numpy as np
import pandas as pd
import holoviews as hv
hv.extension('bokeh')

In [ ]:
df = pd.read_csv('https://fits.geonet.org.nz/observation?siteID=RU001&typeID=t&days=600', parse_dates=['date-time'])

In [ ]:
%%opts Curve  [height=400 width=800 tools=['hover'] ]
simple_curve = hv.Curve(df, 'date-time', ' t (C)')
simple_curve